In [27]:
import cv2
import numpy as np

def compute_disparity(left_image, right_image):
    # 设置SGBM算法的参数
    min_disparity = 0
    num_disparities = 16 * 5
    block_size = 11  # 调整匹配块大小
    P1 = 8 * 3 * block_size**2
    P2 = 32 * 3 * block_size**2
    disp12_max_diff = 1
    uniqueness_ratio = 15
    speckle_window_size = 100
    speckle_range = 32

    # 创建StereoSGBM对象并设置参数
    stereo = cv2.StereoSGBM_create(
        minDisparity=min_disparity,
        numDisparities=num_disparities,
        blockSize=block_size,
        P1=P1,
        P2=P2,
        disp12MaxDiff=disp12_max_diff,
        uniquenessRatio=uniqueness_ratio,
        speckleWindowSize=speckle_window_size,
        speckleRange=speckle_range,
        mode=cv2.STEREO_SGBM_MODE_SGBM
    )

    # 计算视差图
    disparity = stereo.compute(left_image, right_image)
    return disparity

def refine_disparity(disparity):
    # 将视差图转换为8位无符号整数类型
    disparity_scaled = cv2.convertScaleAbs(disparity)

    # 使用中值滤波消除噪声
    disparity_smoothed = cv2.medianBlur(disparity_scaled, 5)

    # 使用二值化处理视差图，过滤掉部分无效区域
    _, binary_disparity = cv2.threshold(disparity_smoothed, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    filtered_disparity = cv2.bitwise_and(disparity_smoothed, binary_disparity)

    return filtered_disparity

def main():
    # 读取左右两张立体图像
    left_image = cv2.imread('left.png', 0)
    right_image = cv2.imread('right.png', 0)

    # 计算视差图
    disparity = compute_disparity(left_image, right_image)

    # 对视差图进行后处理
    refined_disparity = refine_disparity(disparity)

    # 视差图归一化处理
    min_disp = refined_disparity.min()
    max_disp = refined_disparity.max()
    disparity_normalized = ((refined_disparity - min_disp) / (max_disp - min_disp) * 255).astype(np.uint8)

    # 可视化结果
    cv2.imshow('Left Image', left_image)
    cv2.imshow('Right Image', right_image)
    cv2.imshow('Disparity Map', disparity_normalized)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()